## Idea original modificada:
http://philipperemy.github.io/keras-stateful-lstm/

Interestante:

https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/

## Generamos una secuencia artificial para ejemplificar el problema
- Cada muestra es una secuencia binaria de 20 bits. 
- Para X: El primer bit es es 1 con una probabilidad de 0.5, el resto son todos ceros
- Para y: La etiqueta es 1 si el primer bit es 1

In [1]:
import numpy as np
N_samples = 1200
N_bits = 20
from numpy.random import choice
one_indexes = choice(a=N_samples, size=int(N_samples / 2), replace=False)
X = np.zeros((N_samples, N_bits))
X[one_indexes, 0] = 1  # very long term memory.
y = X[:,0]

In [2]:
N_train = 1000
X_train = X[:N_train]
y_train = y[:N_train]
X_validation = X[N_train:]
y_validation = y[N_train:]

In [3]:
idx = 1
print('Entrada: ',X_train[idx])
print('Salida:', y_train[idx])

Entrada:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
Salida: 0.0


In [4]:
print('Training')
print(X_train.shape)
print(y_train.shape)
print('Validation')
print(X_validation.shape)
print(y_validation.shape)

Training
(1000, 20)
(1000,)
Validation
(200, 20)
(200,)


In [5]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

## Que pasa si partimos la secuencia para entrenar?

### Probar:
- window_length = 10, 15, 20
- batch_size = 1, 50, 1000

In [10]:
window_length = 20
X_train_split_1, y_train_split_1 = prepare_sequences(X_train, y_train, window_length)
X_validation_split_1, y_validation_split_1 = prepare_sequences(X_validation, y_validation, window_length)

In [11]:
# Empieza el nuevo vector en (N_bits - window_length + 1)
next_vect_delta = N_bits - window_length + 1
print(X_train_split_1[next_vect_delta])
# Los y's son todos unos si el primer valor del vector fue 1
print(y_train_split_1[next_vect_delta])
print(y_train_split_1[next_vect_delta+1])
print(X_train_split_1.shape)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
0.0
0.0
(1000, 20)


In [12]:
from keras.layers import SimpleRNN, Dense, LSTM
from keras.models import Sequential
batch_size = 50

model = Sequential()
model.add(LSTM(10, input_shape=(window_length, 1), return_sequences=False, stateful=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_split_1.reshape(-1,window_length,1), y_train_split_1, batch_size=batch_size, epochs=15,
          validation_data=(X_validation_split_1.reshape(-1,window_length,1), y_validation_split_1), shuffle=False)
score, acc = model.evaluate(X_validation_split_1.reshape(-1,window_length,1), y_validation_split_1, batch_size=1, verbose=1)
print(score, acc)

Train on 1000 samples, validate on 200 samples
Epoch 1/15
1000/1000 [==============================] - 1s - loss: 0.6931 - acc: 0.5100 - val_loss: 0.6927 - val_acc: 0.4300
Epoch 2/15
1000/1000 [==============================] - 1s - loss: 0.6916 - acc: 0.5140 - val_loss: 0.6938 - val_acc: 0.4300
Epoch 3/15
1000/1000 [==============================] - 1s - loss: 0.6864 - acc: 0.5140 - val_loss: 0.6875 - val_acc: 0.4300
Epoch 4/15
1000/1000 [==============================] - 1s - loss: 0.6393 - acc: 0.5140 - val_loss: 0.5841 - val_acc: 0.4300
Epoch 5/15
1000/1000 [==============================] - 1s - loss: 0.3957 - acc: 0.9240 - val_loss: 0.2931 - val_acc: 1.0000
Epoch 6/15
1000/1000 [==============================] - 1s - loss: 0.2315 - acc: 1.0000 - val_loss: 0.2223 - val_acc: 1.0000
Epoch 7/15
1000/1000 [==============================] - 1s - loss: 0.1823 - acc: 1.0000 - val_loss: 0.1834 - val_acc: 1.0000
Epoch 8/15
1000/1000 [==============================] - 1s - loss: 0.1515 - ac

KeyboardInterrupt: 

## Usando Batch_size de 1, stateful y window_lenght = 1

In [13]:
window_length = 1
X_train_split_2, y_train_split_2 = prepare_sequences(X_train, y_train, window_length)
X_validation_split_2, y_validation_split_2 = prepare_sequences(X_validation, y_validation, window_length)

In [14]:
batch_size = 1

model = Sequential()
model.add(LSTM(10, batch_input_shape=(1, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train_split_2.reshape(-1,window_length,1), y_train_split_2, batch_size=batch_size, epochs=1,
          validation_data=(X_validation_split_2.reshape(-1,window_length,1), y_validation_split_2), shuffle=False)
score, acc = model.evaluate(X_validation_split_2.reshape(-1,window_length,1), y_validation_split_2, batch_size=1, verbose=1)
print(score, acc)

Train on 20000 samples, validate on 4000 samples
Epoch 1/1
3983/4000 [============================>.] - ETA: 0s0.722515078805 0.43


Que pasa en este caso? Por que no llega a mejorar la accuracy?

## Reseteo despues de cada secuencia

In [15]:
i = 1
X_validation_split_2[i*N_bits:(i+1)*N_bits]

array([[ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [189]:
batch_size = 1

model = Sequential()
model.add(LSTM(10, batch_input_shape=(1, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

for i in range(X_train_split_2.shape[0]):
    print(i)
    model.fit(X_train_split_2[i*N_bits:(i+1)*N_bits].reshape(-1,window_length,1), 
              y_train_split_2[i*N_bits:(i+1)*N_bits], batch_size=batch_size, epochs=1, shuffle=False, verbose=1)
    model.reset_states()
score, acc = model.evaluate(X_validation_split_2.reshape(-1,window_length,1), y_validation_split_2, batch_size=1, verbose=1)
print(score, acc)

0
Epoch 1/1
20/20 [==============================] - 1s - loss: 0.6760 - acc: 1.0000     
1
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.7304 - acc: 0.1000     
2
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.6644 - acc: 1.0000     
3
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.7220 - acc: 0.1500     
4
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.6871 - acc: 0.6000     
5
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.6422 - acc: 1.0000     
6
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.7603 - acc: 0.0000e+00     
7
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.7189 - acc: 0.0000e+00     
8
Epoch 1/1
20/20 [==============================] - ETA: 0s - loss: 0.6803 - acc: 0.8889  - 0s - loss: 0.6783 - acc: 0.9000     
9
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.7132 - acc: 0.2000     
10
Epoch 1/1
20/20 [==============================] 

20/20 [==============================] - 0s - loss: 0.0048 - acc: 1.0000     
178
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0046 - acc: 1.0000     
179
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0044 - acc: 1.0000     
180
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0151 - acc: 1.0000     
181
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0043 - acc: 1.0000     
182
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0041 - acc: 1.0000     
183
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0147 - acc: 1.0000     
184
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0143 - acc: 1.0000     
185
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0138 - acc: 1.0000     
186
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0041 - acc: 1.0000     
187
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0133 - acc: 1.0000     
18

20/20 [==============================] - 0s - loss: 3.7243e-04 - acc: 1.0000     
354
Epoch 1/1
20/20 [==============================] - 0s - loss: 3.6304e-04 - acc: 1.0000     
355
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0015 - acc: 1.0000     
356
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0014 - acc: 1.0000     
357
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0014 - acc: 1.0000     - ETA: 0s - loss: 0.0015 - acc: 1.000
358
Epoch 1/1
20/20 [==============================] - 0s - loss: 3.5710e-04 - acc: 1.0000     
359
Epoch 1/1
20/20 [==============================] - 0s - loss: 3.4818e-04 - acc: 1.0000     
360
Epoch 1/1
20/20 [==============================] - 0s - loss: 3.3830e-04 - acc: 1.0000     
361
Epoch 1/1
20/20 [==============================] - 0s - loss: 3.2873e-04 - acc: 1.0000     
362
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0014 - acc: 1.0000     
363
Epoch 1/1
20/20 [============

20/20 [==============================] - 0s - loss: 5.8958e-05 - acc: 1.0000     
525
Epoch 1/1
20/20 [==============================] - 0s - loss: 5.7391e-05 - acc: 1.0000     
526
Epoch 1/1
20/20 [==============================] - 0s - loss: 5.5831e-05 - acc: 1.0000     
527
Epoch 1/1
20/20 [==============================] - 0s - loss: 1.7760e-04 - acc: 1.0000     
528
Epoch 1/1
20/20 [==============================] - 0s - loss: 5.4468e-05 - acc: 1.0000     
529
Epoch 1/1
20/20 [==============================] - 0s - loss: 5.3265e-05 - acc: 1.0000     
530
Epoch 1/1
20/20 [==============================] - 0s - loss: 5.1962e-05 - acc: 1.0000     
531
Epoch 1/1
20/20 [==============================] - 0s - loss: 1.7418e-04 - acc: 1.0000     
532
Epoch 1/1
20/20 [==============================] - 0s - loss: 1.7053e-04 - acc: 1.0000     
533
Epoch 1/1
20/20 [==============================] - 0s - loss: 5.0915e-05 - acc: 1.0000     
534
Epoch 1/1
20/20 [==============================] -

20/20 [==============================] - 0s - loss: 7.0562e-05 - acc: 1.0000     
610
Epoch 1/1
20/20 [==============================] - 0s - loss: 6.9171e-05 - acc: 1.0000     
611
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.3597e-05 - acc: 1.0000     
612
Epoch 1/1
20/20 [==============================] - 0s - loss: 6.7570e-05 - acc: 1.0000     
613
Epoch 1/1
20/20 [==============================] - 0s - loss: 6.6252e-05 - acc: 1.0000     
614
Epoch 1/1
20/20 [==============================] - 0s - loss: 6.4824e-05 - acc: 1.0000     
615
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.3296e-05 - acc: 1.0000     
616
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.2759e-05 - acc: 1.0000     
617
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.2169e-05 - acc: 1.0000     
618
Epoch 1/1
20/20 [==============================] - 0s - loss: 6.3504e-05 - acc: 1.0000     
619
Epoch 1/1
20/20 [==============================] -

20/20 [==============================] - 0s - loss: 4.3334e-06 - acc: 1.0000     
779
Epoch 1/1
20/20 [==============================] - 0s - loss: 1.1369e-05 - acc: 1.0000     
780
Epoch 1/1
20/20 [==============================] - 0s - loss: 1.1156e-05 - acc: 1.0000     
781
Epoch 1/1
20/20 [==============================] - 0s - loss: 4.2500e-06 - acc: 1.0000     
782
Epoch 1/1
20/20 [==============================] - 0s - loss: 4.1665e-06 - acc: 1.0000     
783
Epoch 1/1
20/20 [==============================] - 0s - loss: 1.0917e-05 - acc: 1.0000     
784
Epoch 1/1
20/20 [==============================] - 0s - loss: 1.0712e-05 - acc: 1.0000     
785
Epoch 1/1
20/20 [==============================] - 0s - loss: 4.0831e-06 - acc: 1.0000     
786
Epoch 1/1
20/20 [==============================] - 0s - loss: 4.0056e-06 - acc: 1.0000     
787
Epoch 1/1
20/20 [==============================] - 0s - loss: 1.0482e-05 - acc: 1.0000     
788
Epoch 1/1
20/20 [==============================] -

20/20 [==============================] - 0s - loss: 2.3665e-06 - acc: 1.0000     
949
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.3114e-06 - acc: 1.0000     
950
Epoch 1/1
20/20 [==============================] - 0s - loss: 7.9275e-07 - acc: 1.0000     
951
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.2525e-06 - acc: 1.0000     
952
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.2062e-06 - acc: 1.0000     
953
Epoch 1/1
20/20 [==============================] - 0s - loss: 7.8679e-07 - acc: 1.0000     
954
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.1519e-06 - acc: 1.0000     
955
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.1086e-06 - acc: 1.0000     
956
Epoch 1/1
20/20 [==============================] - 0s - loss: 2.0620e-06 - acc: 1.0000     
957
Epoch 1/1
20/20 [==============================] - 0s - loss: 7.8083e-07 - acc: 1.0000     
958
Epoch 1/1
20/20 [==============================] -

KeyboardInterrupt: 

## Batch size = 1000 y reshape del dataset

In [235]:
batch_size = 1000
X_train_split, y_train_split = prepare_sequences(X_train, y_train, 1)
N = len(X_train_split)
frac = N/batch_size
print(N, frac)
X_train_split_reshaped = X_train_split.reshape(batch_size,int(frac)).T.reshape(N,1,1)
y_train_split_reshaped = y_train_split.reshape(batch_size,int(frac)).T.reshape(N)
print(X_train_split.shape)
print(X_train_split_reshaped.shape)

20000 20.0
(20000, 1)
(20000, 1, 1)


In [236]:
from keras.layers import SimpleRNN, Dense, LSTM
from keras.models import Sequential
model = Sequential()
model.add(LSTM(10, batch_input_shape=(batch_size, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [237]:
for i in range(N_bits):
    model.fit(X_train_split_reshaped, y_train_split_reshaped, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
    model.reset_states()

Epoch 1/1
20000/20000 [==============================] - 1s - loss: 0.6925 - acc: 0.5090     
Epoch 1/1
20000/20000 [==============================] - 0s - loss: 0.6908 - acc: 0.5090     
Epoch 1/1
20000/20000 [==============================] - 0s - loss: 0.6891 - acc: 0.5090     
Epoch 1/1
20000/20000 [==============================] - 0s - loss: 0.6870 - acc: 0.5827     
Epoch 1/1
20000/20000 [==============================] - 0s - loss: 0.6841 - acc: 0.6563     
Epoch 1/1
20000/20000 [==============================] - 0s - loss: 0.6796 - acc: 0.7300     
Epoch 1/1
20000/20000 [==============================] - 0s - loss: 0.6715 - acc: 0.7791     
Epoch 1/1
20000/20000 [==============================] - 0s - loss: 0.6524 - acc: 0.8773     
Epoch 1/1
20000/20000 [==============================] - 0s - loss: 0.5877 - acc: 1.0000     
Epoch 1/1
20000/20000 [==============================] - 0s - loss: 0.4170 - acc: 1.0000     
Epoch 1/1
20000/20000 [==============================] - 0s 

## Para poder evaluar tengo que recargar el modelo con batch_size=200 para que sea eficiente

In [198]:
X_val_split, y_val_split = prepare_sequences(X_validation, y_validation, 1)
X_val_split_reshaped = X_val_split.reshape(200,20).T.reshape(4000,1,1)
y_val_split_reshaped = y_val_split.reshape(200,20).T.reshape(4000)

In [199]:
new_model = Sequential()
new_model.add(LSTM(10, batch_input_shape=(200, 1, 1), return_sequences=False, stateful=True))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [200]:
old_weights = model.get_weights()
new_model.set_weights(old_weights)

In [46]:
new_model.evaluate(X_val_split_reshaped, y_val_split_reshaped, batch_size=200, verbose=1)

 200/4000 [>.............................] - ETA: 2s

[0.20064812153577805, 1.0]

# Batch size = 100

Hay que tomar de a 100 del total de 1000, hacer un reshape a (100,20) y luego trasponer

In [238]:
X_train_split, y_train_split = prepare_sequences(X_train, y_train, 1)

In [240]:
X_train_split.reshape(1000,20)[:100].T #.flatten()

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [206]:
X_train_split.shape

(20000, 1)

In [210]:
X_train_split_reshaped = np.zeros((X_train_split.shape[0]))

In [246]:
batch_size = 100
model = Sequential()
model.add(LSTM(10, batch_input_shape=(batch_size, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [247]:
n_batches = int(len(X_train_split_reshaped)/batch_size)
print(n_batches)
for i in range(10):
    X_to_train = (X_train_split.reshape(1000,20)[i*batch_size:(i+1)*batch_size].T.flatten()).reshape(2000,1,1)
    y_to_train = y_train_split.reshape(1000,20)[i*batch_size:(i+1)*batch_size].T.flatten()
    model.fit(X_to_train, y_to_train, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
    model.reset_states()

200
Epoch 1/1
2000/2000 [==============================] - 1s - loss: 0.6911 - acc: 0.6260     
Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.6930 - acc: 0.5060     
Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.6903 - acc: 0.4965     
Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.6873 - acc: 0.8320     
Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.6832 - acc: 0.6320     
Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.6852 - acc: 0.5320     
Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.6755 - acc: 0.6815     
Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.6628 - acc: 0.9745     
Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.6240 - acc: 1.0000     
Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.4900 - acc: 1.0000     


In [248]:
X_val_split, y_val_split = prepare_sequences(X_validation, y_validation, 1)
X_val_split_reshaped = X_val_split.reshape(200,20).T.reshape(4000,1,1)
y_val_split_reshaped = y_val_split.reshape(200,20).T.reshape(4000)

new_model = Sequential()
new_model.add(LSTM(10, batch_input_shape=(200, 1, 1), return_sequences=False, stateful=True))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

old_weights = model.get_weights()
new_model.set_weights(old_weights)

new_model.evaluate(X_val_split_reshaped, y_val_split_reshaped, batch_size=200, verbose=1)

 200/4000 [>.............................] - ETA: 21s

[0.43642631322145464, 1.0]